In [35]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix
from imblearn.over_sampling import SMOTE
from sklearn.metrics import accuracy_score, recall_score
import warnings
warnings.filterwarnings("ignore")

In [36]:
# Load the training dataset and the test dataset
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')
# Check the data types
print(train_data.dtypes)
# View the first few rows of the dataset
print(train_data.head())

age                int64
workclass         object
fnlwgt             int64
education         object
education-num      int64
marital-status    object
occupation        object
relationship      object
race              object
sex               object
capital-gain       int64
capital-loss       int64
hours-per-week     int64
native-country    object
income            object
dtype: object
   age         workclass  fnlwgt  education  education-num  \
0   39         State-gov   77516  Bachelors             13   
1   50  Self-emp-not-inc   83311  Bachelors             13   
2   38           Private  215646    HS-grad              9   
3   53           Private  234721       11th              7   
4   28           Private  338409  Bachelors             13   

       marital-status         occupation   relationship   race     sex  \
0       Never-married       Adm-clerical  Not-in-family  White    Male   
1  Married-civ-spouse    Exec-managerial        Husband  White    Male   
2            Div

In [37]:
# Convert the 'income' column to numerical values
train_data['income'] = train_data['income'].apply(lambda x: 1 if x == '>50K' else 0)
test_data['income'] = test_data['income'].apply(lambda x: 1 if x == '>50K' else 0)
# Check the data types
print(train_data.dtypes)
# Check the number of missing values in each column
print(train_data.isnull().sum())
# View the unique values in the 'race' column
unique_race_values = train_data['race'].unique()
print(unique_race_values)
# View the unique values in the 'sex' column
unique_sex_values = train_data['sex'].unique()
print(unique_sex_values)

age                int64
workclass         object
fnlwgt             int64
education         object
education-num      int64
marital-status    object
occupation        object
relationship      object
race              object
sex               object
capital-gain       int64
capital-loss       int64
hours-per-week     int64
native-country    object
income             int64
dtype: object
age               0
workclass         0
fnlwgt            0
education         0
education-num     0
marital-status    0
occupation        0
relationship      0
race              0
sex               0
capital-gain      0
capital-loss      0
hours-per-week    0
native-country    0
income            0
dtype: int64
['White' 'Black' 'Asian-Pac-Islander' 'Amer-Indian-Eskimo' 'Other']
['Male' 'Female']


In [38]:
# Define numerical and categorical features
numeric_features = ['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week']
categorical_features = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country']

# Data preprocessing: Standardize numerical features and one-hot encode categorical features
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ],
    remainder='passthrough'
)

# Create a model pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(max_iter=1000))
])

In [39]:
# Prepare the training and testing data
X_train = train_data.drop(columns=['income'])
y_train = train_data['income']
X_test = test_data.drop(columns=['income'])
y_test = test_data['income']

### baseline model

In [40]:
def calculate_metrics(y_true, y_pred, sensitive_attribute, data):
    overall_accuracy = accuracy_score(y_true, y_pred)
    print(f"Overall Accuracy: {overall_accuracy:.4f}")
    
    groups = data[sensitive_attribute].unique()
    for group in groups:
        group_mask = data[sensitive_attribute] == group
        # TPR = TP / (TP + FN)
        group_tpr = recall_score(y_true[group_mask], y_pred[group_mask])
        print(f"TPR for {sensitive_attribute}={group}: {group_tpr:.4f}")

pipeline.fit(X_train, y_train)
y_pred_baseline = pipeline.predict(X_test)
accuracy_baseline = accuracy_score(y_test, y_pred_baseline)
print(f"Baseline Accuracy: {accuracy_baseline:.4f}")
print("Baseline Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_baseline))
print("Baseline Fairness Metrics for Sex (TPR):")
calculate_metrics(y_test, y_pred_baseline, 'sex', test_data)


Baseline Accuracy: 0.8530
Baseline Confusion Matrix:
[[11586   849]
 [ 1544  2302]]
Baseline Fairness Metrics for Sex (TPR):
Overall Accuracy: 0.8530
TPR for sex=Male: 0.6118
TPR for sex=Female: 0.5254


### data Intervention

In [41]:
# [Data Exploration] Check the number of samples for each gender in the training set
print("Gender distribution in the original training set:")
print(train_data['sex'].value_counts())

Gender distribution in the original training set:
sex
Male      21790
Female    10771
Name: count, dtype: int64


### Controlled Sampling

In [42]:
# %% 
# [Intervention Method 1: Controlled Sampling (Improved Version)]
# The goal is to adjust the gender ratio in the new training set to 55% males and 45% females (without changing the total number of samples).
# %%

import random
from collections import defaultdict

def p2data(data):
    """
    Intervention at the data level:
    Perform random oversampling on the two groups (z=True and z=False) to achieve the target ratio while balancing positive and negative labels within each group.

    data: list of (d, z, l), where z is 0/1 (0 for Female, 1 for Male), and l is the label (0/1)
    return: list of (d, z, l) (processed data)
    """
    target_z_ratio = 1.8165  # Target ratio: z=False / z=True
    # Group by z and label
    groups = defaultdict(list)
    for d, z, l in data:
        groups[(z, l)].append((d, z, l))

    # Original counts for each group
    group_counts = {k: len(v) for k, v in groups.items()}

    # Limit the maximum sampling multiple to prevent excessive oversampling
    max_size = max(group_counts.values()) * 5

    # Calculate the total number of samples for each z group (regardless of label)
    z0_total = group_counts.get((0, True), 0) + group_counts.get((0, False), 0)
    z1_total = group_counts.get((1, True), 0) + group_counts.get((1, False), 0)

    # Control the overall distribution of z
    desired_z0 = min(int(z1_total * target_z_ratio), max_size)
    desired_z1 = min(int(z0_total / target_z_ratio), max_size)

    def balance_group(pos_count, neg_count, data_pos, data_neg):
        # If either label group is empty, simply merge them
        if pos_count == 0 or neg_count == 0:
            return data_pos + data_neg
        # Determine the majority and minority groups
        if pos_count < neg_count:
            major, minor = data_neg, data_pos
        else:
            major, minor = data_pos, data_neg
        # Oversample the minority class
        times = len(major) // len(minor)
        remainder = len(major) % len(minor)
        sampled = minor * times + random.sample(minor, remainder)
        return sampled + major

    # Process the z==0 group (corresponding to Female)
    balanced_z0 = balance_group(
        group_counts.get((0, True), 0),
        group_counts.get((0, False), 0),
        groups.get((0, True), []),
        groups.get((0, False), [])
    )
    balanced_z0 = random.sample(balanced_z0, min(desired_z0, len(balanced_z0)))

    # Process the z==1 group (corresponding to Male)
    balanced_z1 = balance_group(
        group_counts.get((1, True), 0),
        group_counts.get((1, False), 0),
        groups.get((1, True), []),
        groups.get((1, False), [])
    )
    balanced_z1 = random.sample(balanced_z1, min(desired_z1, len(balanced_z1)))

    balanced_data = balanced_z0 + balanced_z1
    random.shuffle(balanced_data)
    return balanced_data

# Convert DataFrame to a list of (d, z, l); here d is the entire row data
def df2list(df):
    data_list = []
    for _, row in df.iterrows():
        # Convention: Female -> z=0, Male -> z=1
        z = 0 if row['sex'] == 'Female' else 1
        l = row['income']
        # Pass the entire row directly (use pd.DataFrame(list_of_d) when converting back to DataFrame)
        data_list.append((row.to_dict(), z, l))
    return data_list

def list2df(data_list):
    # Extract the d dictionary and construct a DataFrame
    return pd.DataFrame([d for d, z, l in data_list])

# Apply sampling intervention to the training data
data_list = df2list(train_data)
balanced_data_list = p2data(data_list)
train_data_balanced = list2df(balanced_data_list)

print("Sample counts for each gender after intervention:")
print(train_data_balanced['sex'].value_counts())

# Train the model using the balanced data
X_train_bal = train_data_balanced.drop(columns=['income'])
y_train_bal = train_data_balanced['income']

pipeline.fit(X_train_bal, y_train_bal)
y_pred_bal = pipeline.predict(X_test)
accuracy_bal = accuracy_score(y_test, y_pred_bal)
print(f"Model accuracy after sampling intervention: {accuracy_bal:.4f}")
print("Confusion matrix after sampling intervention:")
print(confusion_matrix(y_test, y_pred_bal))
print("Fairness metrics for gender (TPR) after sampling intervention:")
calculate_metrics(y_test, y_pred_bal, 'sex', test_data)

Sample counts for each gender after intervention:
sex
Female    19184
Male       5929
Name: count, dtype: int64
Model accuracy after sampling intervention: 0.8094
Confusion matrix after sampling intervention:
[[10072  2363]
 [  740  3106]]
Fairness metrics for gender (TPR) after sampling intervention:
Overall Accuracy: 0.8094
TPR for sex=Male: 0.8034
TPR for sex=Female: 0.8305


### Model Weight Adjustment

In [43]:
# %% 
# [Intervention Method 2: Model Weight Adjustment]
# Use the `class_weight` parameter of LogisticRegression to automatically adjust the weights of different classes (income) to mitigate the impact of class imbalance on model training, thereby indirectly improving the performance of different sensitive groups (e.g., gender).
# Note: Here, `class_weight` adjusts the weights of the income labels, which is a common in-processing intervention method.

# %%

from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

# Calculate dynamic weights: Add an auxiliary column 'z' to X_train, mapping 'Female' to 0 and 'Male' to 1
X_train_weight = X_train.copy()
X_train_weight['z'] = X_train_weight['sex'].apply(lambda s: 0 if s == 'Female' else 1)

# Count the number of positive examples in each group (z==0 for Female, z==1 for Male)
z0_pos = sum((X_train_weight['z'] == 0) & (y_train == 1))
z1_pos = sum((X_train_weight['z'] == 1) & (y_train == 1))

# Calculate dynamic weights based on the sensitive attribute and label of each sample
weights = []
for i, row in X_train_weight.iterrows():
    z_val = row['z']
    l_val = y_train.iloc[i]
    # Dynamic weight calculation logic: Increase the weight of positive examples for the disadvantaged group (Female, z==0), and slightly suppress the weight for the advantaged group (Male, z==1)
    if z_val == 0:  # Disadvantaged group (Female)
        weight = 5.2 if l_val == 1 else 3.2
    else:           # Advantaged group (Male)
        weight = 1 if l_val == 1 else 1
    # Automatically adjust based on the total number of positive examples (to prevent excessive weight differences)
    if z_val == 0:
        weight *= (z0_pos + z1_pos) / (z0_pos + 1e-5)
    else:
        weight *= (z0_pos + z1_pos) / (z1_pos + 1e-5)
    weights.append(weight)

# Construct the model pipeline using the preprocessor and LogisticRegression classifier
model_dynamic = Pipeline([
    ('preprocessor', preprocessor),  # Preprocessor: Standardize numerical features and one-hot encode categorical features
    ('classifier', LogisticRegression(max_iter=1000, random_state=42))
])

# Train the model with the dynamically calculated sample_weight
model_dynamic.fit(X_train, y_train, classifier__sample_weight=weights)
y_pred_dynamic = model_dynamic.predict(X_test)

accuracy_dynamic = accuracy_score(y_test, y_pred_dynamic)
print(f"Model accuracy with dynamic weight adjustment: {accuracy_dynamic:.4f}")
print("Confusion matrix with dynamic weight adjustment:")
print(confusion_matrix(y_test, y_pred_dynamic))
print("Fairness metrics for gender (TPR) with dynamic weight adjustment:")
calculate_metrics(y_test, y_pred_dynamic, 'sex', test_data)

Model accuracy with dynamic weight adjustment: 0.8485
Confusion matrix with dynamic weight adjustment:
[[11455   980]
 [ 1486  2360]]
Fairness metrics for gender (TPR) with dynamic weight adjustment:
Overall Accuracy: 0.8485
TPR for sex=Male: 0.6139
TPR for sex=Female: 0.6119


### Reweighting with Smoothing(Based on the paper)

In [44]:
# %% 
# [Intervention Method 3: Reweighting with Smoothing]
# Based on the formula proposed by Kamiran and Calders (2011):
# [Improved Reweighting Method: Introducing a Smoothing Parameter]
# Original formula: weight = (n_{sex} * n_{income}) / (n_total * n_{sex,income})
# To avoid over-adjustment (which may significantly reduce the performance for males while overly boosting it for females), 
# we apply a smoothing technique to the original weights using interpolation: 
# new_weight = (1 - lambda) * 1 + lambda * original_weight, 
# where lambda is the smoothing parameter (0 < lambda < 1). A smaller lambda makes the new weights closer to 1 (i.e., no reweighting effect), 
# while a larger lambda makes the new weights closer to the original values. You can adjust lambda_param to balance overall performance and fairness.

lambda_param = 0.3  # Adjustable parameter: e.g., 0.5 means the new weight is the average of the original weight and 1
n = len(train_data)
group_counts = train_data.groupby('sex').size().to_dict()       # n_{sex}
label_counts = train_data.groupby('income').size().to_dict()      # n_{income}
joint_counts = train_data.groupby(['sex', 'income']).size().to_dict()  # n_{sex,income}

def compute_weight(sex, income):
    n_a = group_counts[sex]
    n_y = label_counts[income]
    n_a_y = joint_counts[(sex, income)]
    return (n_a * n_y) / (n * n_a_y)
# Calculate original weights
sample_weights = train_data.apply(lambda row: compute_weight(row['sex'], row['income']), axis=1)

# Smoothing: Adjusted weights
adjusted_weights = (1 - lambda_param) + lambda_param * sample_weights

print("Adjusted Reweighting: Average smoothed weights for each gender group:")
print(train_data.groupby('sex').apply(
    lambda df: np.mean((1 - lambda_param) + lambda_param * df.apply(lambda row: compute_weight(row['sex'], row['income']), axis=1))
))

# Train the model using the adjusted weights
pipeline.fit(X_train, y_train, classifier__sample_weight=adjusted_weights)
y_pred_rew_adjusted = pipeline.predict(X_test)
accuracy_rew_adjusted = accuracy_score(y_test, y_pred_rew_adjusted)
print(f"Adjusted Reweighting Model Accuracy: {accuracy_rew_adjusted:.4f}")
print("Adjusted Reweighting Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_rew_adjusted))
print("Adjusted Reweighting Fairness Metrics for Sex (TPR):")
calculate_metrics(y_test, y_pred_rew_adjusted, 'sex', test_data)

Adjusted Reweighting: Average smoothed weights for each gender group:
sex
Female    1.0
Male      1.0
dtype: float64
Adjusted Reweighting Model Accuracy: 0.8509
Adjusted Reweighting Confusion Matrix:
[[11597   838]
 [ 1590  2256]]
Adjusted Reweighting Fairness Metrics for Sex (TPR):
Overall Accuracy: 0.8509
TPR for sex=Male: 0.5866
TPR for sex=Female: 0.5864


###  Massaging(Based on the paper)

In [45]:
# %% 
# [Intervention Method 4: Massaging]
# Based on the massaging technique by Kamiran and Calders (2011): Flip the labels of some samples near the decision boundary to reduce the gap in positive example ratios between different gender groups while maintaining model performance as much as possible.
# 1. Train a logistic regression model on the original data to calculate the positive class probabilities (scores) for each sample.
# 2. For the unprivileged group (Female), flip the labels of negative examples with higher scores; for the privileged group (Male), flip the labels of positive examples with lower scores.
# %%

# [Improved Massaging Method: Introducing the Flip Ratio Parameter alpha]
# To avoid too many positive examples in the unprivileged group (Female), we introduce a parameter alpha (0 < alpha <= 1) to control the actual number of samples to be flipped after calculating the original number of flips (delta).
# A smaller alpha means gentler flipping; for example, alpha = 0.5 means flipping only half of the originally required number.

alpha = 0.068  # Flip ratio parameter, adjustable based on experimental results

# Calculate the original number of flips (delta) as before
X_train_trans = preprocessor.fit_transform(X_train)
lr_massaging = LogisticRegression(max_iter=1000, random_state=42)
lr_massaging.fit(X_train_trans, y_train)
scores = lr_massaging.predict_proba(X_train_trans)[:, 1]

# raw data、score and label
train_massaging = X_train.copy()
train_massaging['income'] = y_train.values
train_massaging['score'] = scores
privileged = 'Male'
unprivileged = 'Female'
n_priv_pos = train_massaging[train_massaging['sex'] == privileged]['income'].sum()
n_unpriv_pos = train_massaging[train_massaging['sex'] == unprivileged]['income'].sum()
delta = int(abs(n_priv_pos - n_unpriv_pos) / 2)
n_priv_pos = train_massaging[train_massaging['sex'] == privileged]['income'].sum()
n_unpriv_pos = train_massaging[train_massaging['sex'] == unprivileged]['income'].sum()
delta = int(abs(n_priv_pos - n_unpriv_pos) / 2)


# Actual number of flips
new_delta = int(alpha * delta)
print(f"Massaging: Original number of labels to flip (delta): {delta}, Adjusted number of flips: {new_delta}")

# For Female (unprivileged) negative examples: Select the top new_delta candidates with the highest scores for flipping
candidates_unpriv = train_massaging[(train_massaging['sex'] == unprivileged) & (train_massaging['income'] == 0)]
candidates_unpriv = candidates_unpriv.sort_values(by='score', ascending=False)
indices_to_flip_unpriv = candidates_unpriv.index[:new_delta]

# For Male (privileged) positive examples: Select the bottom new_delta candidates with the lowest scores for flipping
candidates_priv = train_massaging[(train_massaging['sex'] == privileged) & (train_massaging['income'] == 1)]
candidates_priv = candidates_priv.sort_values(by='score', ascending=True)
indices_to_flip_priv = candidates_priv.index[:new_delta]

# Flip labels: Change negative examples in unprivileged group to positive, and positive examples in privileged group to negative
y_train_massaged_adj = y_train.copy()
y_train_massaged_adj.loc[indices_to_flip_unpriv] = 1
y_train_massaged_adj.loc[indices_to_flip_priv] = 0

print("Adjusted Massaging: Comparison of positive example counts before and after flipping:")
print("Female: Original positive count =", train_massaging[train_massaging['sex'] == unprivileged]['income'].sum(),
      "  After change =", y_train_massaged_adj[train_massaging['sex'] == unprivileged].sum())
print("Male: Original positive count =", train_massaging[train_massaging['sex'] == privileged]['income'].sum(),
      "  After change =", y_train_massaged_adj[train_massaging['sex'] == privileged].sum())

# Train the model with the adjusted labels
pipeline.fit(X_train, y_train_massaged_adj)
y_pred_massaged_adj = pipeline.predict(X_test)
accuracy_massaged_adj = accuracy_score(y_test, y_pred_massaged_adj)
print(f"Adjusted Massaging Model Accuracy: {accuracy_massaged_adj:.4f}")
print("Adjusted Massaging Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_massaged_adj))
print("Adjusted Massaging Fairness Metrics for Sex (TPR):")
calculate_metrics(y_test, y_pred_massaged_adj, 'sex', test_data)

Massaging: Original number of labels to flip (delta): 2741, Adjusted number of flips: 186
Adjusted Massaging: Comparison of positive example counts before and after flipping:
Female: Original positive count = 1179   After change = 1365
Male: Original positive count = 6662   After change = 6476
Adjusted Massaging Model Accuracy: 0.8506
Adjusted Massaging Confusion Matrix:
[[11495   940]
 [ 1493  2353]]
Adjusted Massaging Fairness Metrics for Sex (TPR):
Overall Accuracy: 0.8506
TPR for sex=Male: 0.6121
TPR for sex=Female: 0.6102


In [46]:
def get_group_tpr(y_true, y_pred, group_value, sensitive_attribute='sex'):
    mask = test_data[sensitive_attribute] == group_value
    return recall_score(y_true[mask], y_pred[mask])

baseline_male_tpr = get_group_tpr(y_test, y_pred_baseline, 'Male')
baseline_female_tpr = get_group_tpr(y_test, y_pred_baseline, 'Female')

res_male_tpr = get_group_tpr(y_test, y_pred_bal, 'Male')
res_female_tpr = get_group_tpr(y_test, y_pred_bal, 'Female')

dynamic_male_tpr = get_group_tpr(y_test, y_pred_dynamic, 'Male')
dynamic_female_tpr = get_group_tpr(y_test, y_pred_dynamic, 'Female')

rew_male_tpr = get_group_tpr(y_test, y_pred_rew_adjusted, 'Male')
rew_female_tpr = get_group_tpr(y_test, y_pred_rew_adjusted, 'Female')

massaged_male_tpr = get_group_tpr(y_test, y_pred_massaged_adj, 'Male')
massaged_female_tpr = get_group_tpr(y_test, y_pred_massaged_adj, 'Female')

results = {
    'Method': ['Baseline', 'Controlled Resampling', 'Dynamic Weight Adjustment', 'Adjusted Reweighting', 'Adjusted Massaging'],
    'Accuracy': [accuracy_baseline, accuracy_bal, accuracy_dynamic, accuracy_rew_adjusted, accuracy_massaged_adj],
    'Male TPR': [baseline_male_tpr, res_male_tpr, dynamic_male_tpr, rew_male_tpr, massaged_male_tpr],
    'Female TPR': [baseline_female_tpr, res_female_tpr, dynamic_female_tpr, rew_female_tpr, massaged_female_tpr]
}

results_df = pd.DataFrame(results)
print("Overall accuracy and TPR comparison for males and females across all intervention methods:")
print(results_df)


Overall accuracy and TPR comparison for males and females across all intervention methods:
                      Method  Accuracy  Male TPR  Female TPR
0                   Baseline  0.853019  0.611794    0.525424
1      Controlled Resampling  0.809410  0.803440    0.830508
2  Dynamic Weight Adjustment  0.848535  0.613943    0.611864
3       Adjusted Reweighting  0.850869  0.586609    0.586441
4         Adjusted Massaging  0.850562  0.612101    0.610169
